## This notebook takes all the cells detected in the previous two notebooks and combines information, linking the Calcium signals and FRET signal for each neuron, showcasing both. Neurons expressing both Calcium and FRET signals will be saved in a summary Excel file
It should be run after processing the FRET data in 1a_ProcessingFRET.ipynb and calcium imaging data in 1b_ProcessingCalcium.ipynb


In [2]:
import os
from PIL import Image
import numpy as np
import pandas as pd
import pickle


### First import cell showing a good FRET signal

In [3]:
infile = open('FRETgood_cells.pkl', 'rb')
FRET_cells= pickle.load(infile)
infile.close()


### Import information from the mask used to compute the calcium signal from FIJI

In [4]:


# Define the path to the directory containing RCaMP masks
path_mask_rcamp = r'C:\Users\SimonZ\Desktop\Code\graffproject\calciumrec\masks'

# List to store the paths of RCaMP mask files
list_of_rcamp_files = []

# Traverse the directory structure to find RCaMP mask files
for root, dirs, files in os.walk(path_mask_rcamp):
    for file in files:
        # Construct the full path of the RCaMP mask file
        full_path = os.path.join(root, file)
        
        # Append the full path to the list of RCaMP mask files
        list_of_rcamp_files.append(full_path)

# Print the paths of RCaMP mask files
for file_path in list_of_rcamp_files:
    print(file_path)


C:\Users\SimonZ\Desktop\Code\graffproject\calciumrec\masks\01.06_H3_Well3_RCampMask.tif
C:\Users\SimonZ\Desktop\Code\graffproject\calciumrec\masks\31.05_H3_Well5_RCampMask.tif
C:\Users\SimonZ\Desktop\Code\graffproject\calciumrec\masks\31.05_H3_Well6_RCampMask.tif


In [5]:
# List to store the masks for which a corresponding neuron is found
mask_attributed = []

# Iterate over all FRET cells
for cell in FRET_cells:
    # Get coordinates from the information in the FRET cell
    coordinate = cell["Coordinates"]
    sep = cell["Label"].split('_')
    name_rec = sep[0] + '_' + sep[1] + '_' + sep[2]

    # Find the corresponding RCAMP mask file
    corresponding_mask_file = None
    for mask_file in list_of_rcamp_files:
        if name_rec in mask_file:
            corresponding_mask_file = mask_file

    # Open and process the mask file
    mask = Image.open(corresponding_mask_file)
    array_maskfile = np.array(mask)
    df_maskfile = pd.DataFrame(array_maskfile)
    
    # Convert FRET file coordinates to mask file coordinates
    img_size = 2048 * 0.3028823  # Image size in um
    pixel = 256.0  # Number of pixels per row/column

    x = int(coordinate[0] * pixel / img_size)
    y = int(coordinate[1] * pixel / img_size)

    # Check if a neuron is found for this mask
    if df_maskfile.iloc[y, x] != 0:
        mask_attributed.append(df_maskfile.iloc[y, x])
        cell["CaNumber"] = df_maskfile.iloc[y, x]
    else:
        mask_attributed.append(0)
        cell["CaNumber"] = 0

# Count the number of nuclei found with a corresponding mask
nb_nucleus_wo_mask = len([mask for mask in mask_attributed if mask != 0])
nb_nucleus = len(mask_attributed)

print("Out of", nb_nucleus, "cells,", nb_nucleus_wo_mask, "found a corresponding mask")


Out of 125 cells, 115 found a corresponding mask


### Import calcium signal and find cells that are positive both for calcium and FRET

In [6]:
infile = open('CArobust_cells.pkl', 'rb')
Ca_cells= pickle.load(infile)
infile.close()


In [7]:
double_positive_cells =[]
# Iterate over the FRET cells
for cell in FRET_cells:
    corresponding_mask_number = cell["CaNumber"]

    if corresponding_mask_number!= 0: #a corresponding mask has been found

        name = cell["Label"]
        sep = name.split('_')
        corresponding_ca_name = sep[0]+'_'+sep[2]+'_'+sep[1]+'_'+sep[-1] # I messed up the naming convention, this trick is necessary to find the match 
        
        for Cacell in Ca_cells: # Iterate over all the cells in the other file and find a match (if it exists!)
            cell_ca = Cacell["Name"].split('_')
            cell_ca_name = cell_ca[0]+'_'+cell_ca[1]+'_'+cell_ca[2] 
            cell_ca_nb =  Cacell["Name"].split('_')[-1]

            if cell_ca_name in corresponding_ca_name and cell_ca_nb == str(int(corresponding_mask_number)): #If the recording name are the same, and the number is the same
                Cacell["FRET"]=cell
                double_positive_cells.append(Cacell)

print("In total:",len(double_positive_cells),"double positive cells has been found")


In total: 53 double positive cells has been found


In [8]:
def nb_calcium_event_in_timebins(Cell, timebins):
    """
    Calculate the number of calcium events in each time bin for a given cell.

    Parameters:
        Cell (dict): Dictionary containing information about the cell, including the experimental interval indices.
        timebins (list): List of time bin boundaries.

    Returns:
        numpy.ndarray: Array containing the number of calcium events in each time bin.
    """
    # Initialize an array to store the number of calcium events in each time bin
    active_in_bin = np.zeros(len(timebins) - 1)
    
    # Extract the indices of calcium events within the experimental interval
    indx = Cell['Experimental_Interval']['index']
    
    # Iterate over each time bin
    for i in range(len(timebins) - 1):
        # Count the number of calcium events within each time bin
        active_in_bin[i] = len(indx[(indx >= timebins[i]) & (indx <= timebins[i + 1])])

    return active_in_bin


### We computed the number of calcium events occuring in each 20 sec time bin during the experimental period of the recording

In [9]:

timebins_used = np.linspace(1909,6000,46)

for cell in double_positive_cells:
    activity=nb_calcium_event_in_timebins(cell,timebins_used)
    cell["Activity"]=activity
    

### Create an excel with a summary of the activity of each neurons and the corresponding NFRET value and low-mid-high class

In [10]:
# Define the column names for the DataFrame
col = ["LabelFRET", "Rep", "FRETvalue"]

biological_replicate = "First"  # Assuming it's the first biological replicate

# Iterate over time bins to create column names for each time frame
for i in range(len(timebins_used) - 1):
    col.append("Fr" + str(int(timebins_used[i])) + '-' + str(int(timebins_used[i + 1])))

# Create an empty list to store dictionaries representing rows of data
data_rows = []

# Iterate over double positive cells to populate the list of data rows
for cell in double_positive_cells:
    # Extract class FRET and biological replicate information
    class_FRET = cell["FRET"]["class"]
    nfret = cell["FRET"]["meanval"]['NFRET']
    # Create a dictionary to store cell activity data
    dic = {"LabelFRET": class_FRET, "Rep": biological_replicate,"FRETvalue":nfret}
    
    # Populate the dictionary with cell activity data for each time frame
    for i in range(len(cell["Activity"])):
        dic[col[i + 3]] = cell["Activity"][i] + 1  # Start adding from the 3rd column onwards,
        ## NB the +1 is an adjustment needed for our computation, think of removing 1 event by time bins when you perform your operation!
        
    # Append the dictionary to the list of data rows
    data_rows.append(dic)

# Create a DataFrame from the list of data rows
df = pd.DataFrame(data_rows, columns=col)

# Define the file name for the Excel file
excel_file_name = "Summary_"+biological_replicate+"_BiologicalReplicates.xlsx"

# Save the DataFrame to an Excel file
df.to_excel(excel_file_name)



In [11]:
df

,LabelFRET,Rep,FRETvalue,Fr1909-1999,Fr1999-2090,Fr2090-2181,Fr2181-2272,Fr2272-2363,Fr2363-2454,Fr2454-2545,...,Fr5090-5181,Fr5181-5272,Fr5272-5363,Fr5363-5454,Fr5454-5545,Fr5545-5636,Fr5636-5727,Fr5727-5818,Fr5818-5909,Fr5909-6000
0,Low,First,3.796984,4.0,5.0,5.0,5.0,4.0,5.0,4.0,...,3.0,2.0,2.0,4.0,3.0,4.0,3.0,2.0,4.0,3.0
1,Mid,First,4.588951,2.0,1.0,1.0,2.0,1.0,1.0,2.0,...,1.0,1.0,2.0,1.0,1.0,2.0,1.0,2.0,1.0,3.0
2,High,First,12.377859,1.0,1.0,4.0,4.0,4.0,4.0,3.0,...,3.0,4.0,4.0,4.0,2.0,4.0,4.0,3.0,3.0,3.0
3,Low,First,0.695981,2.0,3.0,3.0,3.0,3.0,3.0,3.0,...,1.0,2.0,1.0,1.0,2.0,1.0,2.0,1.0,1.0,1.0
4,High,First,12.551569,1.0,1.0,2.0,1.0,2.0,1.0,1.0,...,1.0,1.0,2.0,1.0,1.0,2.0,1.0,1.0,1.0,3.0
5,High,First,7.987392,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
6,Mid,First,6.966807,1.0,1.0,2.0,1.0,2.0,1.0,1.0,...,1.0,1.0,1.0,2.0,1.0,2.0,1.0,1.0,2.0,1.0
7,High,First,12.140093,2.0,2.0,2.0,2.0,2.0,2.0,2.0,...,1.0,2.0,2.0,1.0,2.0,2.0,2.0,1.0,2.0,1.0
8,Mid,First,6.365438,1.0,3.0,1.0,1.0,2.0,2.0,2.0,...,2.0,3.0,2.0,4.0,3.0,2.0,2.0,3.0,2.0,2.0
9,High,First,12.051530,1.0,1.0,1.0,2.0,1.0,1.0,1.0,...,1.0,1.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,2.0
